In [1]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import ModelFamily

In [2]:
def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [3]:
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     # api_key="YOUR_API_KEY",
# )
model_client = OpenAIChatCompletionClient(
    model="qwen2.5-coder:14b",
    # model="mistral-nemo:12b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.R1,
    },
)

travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [4]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

In [5]:
task = "I need to refund my flight."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task), output_stats=True)
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


# Use asyncio.run(...) if you are running this in a script.
await run_team_stream()

---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
Sure! To help you with the refund process, could you please provide me with your booking confirmation number and the reason for the refund?
[Prompt tokens: 231, Completion tokens: 28]
---------- travel_agent ----------



/Users/kevinlights/app/miniforge3/envs/open_manus/lib/python3.12/site-packages/autogen_ext/models/openai/_openai_client.py:608: UserWarning: Could not find <think>..</think> field in model response content. No thought was extracted.
  thought, content = parse_r1_content(content)


[Prompt tokens: 258, Completion tokens: 1]
---------- travel_agent ----------
Once you provide this information, I can assist you further or transfer you directly to the flights_refunder if needed.

Your details:
- Booking Confirmation Number:
- Reason for Refund:

Please let me know!
[Prompt tokens: 258, Completion tokens: 43]
---------- travel_agent ----------

[Prompt tokens: 300, Completion tokens: 1]
---------- travel_agent ----------
[FunctionCall(id='call_monp5lio', arguments='{}', name='transfer_to_user')]
[Prompt tokens: 300, Completion tokens: 14]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', name='transfer_to_user', call_id='call_monp5lio', is_error=False)]
---------- travel_agent ----------
Transferred to user, adopting the role of user immediately.
---------- Summary ----------
Number of messages: 8
Finish reason: Handoff to user from travel_agent detected.
Total prompt tokens: 1347
Total 

In [6]:
async def get_stock_data(symbol: str) -> Dict[str, Any]:
    """Get stock market data for a given symbol"""
    return {"price": 180.25, "volume": 1000000, "pe_ratio": 65.4, "market_cap": "700B"}


async def get_news(query: str) -> List[Dict[str, str]]:
    """Get recent news articles about a company"""
    return [
        {
            "title": "Tesla Expands Cybertruck Production",
            "date": "2024-03-20",
            "summary": "Tesla ramps up Cybertruck manufacturing capacity at Gigafactory Texas, aiming to meet strong demand.",
        },
        {
            "title": "Tesla FSD Beta Shows Promise",
            "date": "2024-03-19",
            "summary": "Latest Full Self-Driving beta demonstrates significant improvements in urban navigation and safety features.",
        },
        {
            "title": "Model Y Dominates Global EV Sales",
            "date": "2024-03-18",
            "summary": "Tesla's Model Y becomes best-selling electric vehicle worldwide, capturing significant market share.",
        },
    ]

In [7]:
# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     # api_key="YOUR_API_KEY",
# )

model_client = OpenAIChatCompletionClient(
    model="qwen2.5-coder:14b",
    # model="mistral-nemo:12b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
)

planner = AssistantAgent(
    "planner",
    model_client=model_client,
    handoffs=["financial_analyst", "news_analyst", "writer"],
    system_message="""You are a research planning coordinator.
    Coordinate market research by delegating to specialized agents:
    - Financial Analyst: For stock data analysis
    - News Analyst: For news gathering and analysis
    - Writer: For compiling final report
    Always send your plan first, then handoff to appropriate agent.
    Always handoff to a single agent at a time.
    Use TERMINATE when research is complete.""",
)

financial_analyst = AssistantAgent(
    "financial_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_stock_data],
    system_message="""You are a financial analyst.
    Analyze stock market data using the get_stock_data tool.
    Provide insights on financial metrics.
    Always handoff back to planner when analysis is complete.""",
)

news_analyst = AssistantAgent(
    "news_analyst",
    model_client=model_client,
    handoffs=["planner"],
    tools=[get_news],
    system_message="""You are a news analyst.
    Gather and analyze relevant news using the get_news tool.
    Summarize key market insights from news.
    Always handoff back to planner when analysis is complete.""",
)

writer = AssistantAgent(
    "writer",
    model_client=model_client,
    handoffs=["planner"],
    system_message="""You are a financial report writer.
    Compile research findings into clear, concise reports.
    Always handoff back to planner when writing is complete.""",
)

In [8]:
# Define termination condition
text_termination = TextMentionTermination("TERMINATE")
termination = text_termination

research_team = Swarm(
    participants=[planner, financial_analyst, news_analyst, writer], termination_condition=termination
)

task = "Conduct market research for TSLA stock"
await Console(research_team.run_stream(task=task), output_stats=True)

---------- user ----------
Conduct market research for TSLA stock
---------- planner ----------
[FunctionCall(id='call_jr0f1etj', arguments='{}', name='transfer_to_financial_analyst')]
[Prompt tokens: 306, Completion tokens: 40]
---------- planner ----------
[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', name='transfer_to_financial_analyst', call_id='call_jr0f1etj', is_error=False)]
---------- planner ----------
Transferred to financial_analyst, adopting the role of financial_analyst immediately.
---------- financial_analyst ----------
[FunctionCall(id='call_lkvwfa3c', arguments='{"symbol":"TSLA"}', name='get_stock_data')]
[Prompt tokens: 245, Completion tokens: 50]
---------- financial_analyst ----------
[FunctionExecutionResult(content="{'price': 180.25, 'volume': 1000000, 'pe_ratio': 65.4, 'market_cap': '700B'}", name='get_stock_data', call_id='call_lkvwfa3c', is_error=False)]
---------- financial_analyst ---

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Conduct market research for TSLA stock', type='TextMessage'), ToolCallRequestEvent(source='planner', models_usage=RequestUsage(prompt_tokens=306, completion_tokens=40), metadata={}, content=[FunctionCall(id='call_jr0f1etj', arguments='{}', name='transfer_to_financial_analyst')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='planner', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', name='transfer_to_financial_analyst', call_id='call_jr0f1etj', is_error=False)], type='ToolCallExecutionEvent'), HandoffMessage(source='planner', models_usage=None, metadata={}, target='financial_analyst', content='Transferred to financial_analyst, adopting the role of financial_analyst immediately.', context=[], type='HandoffMessage'), ToolCallRequestEvent(source='financial_analyst', models_usa